<a href="https://colab.research.google.com/github/Hushpuppyzac/DLI-Assignment/blob/main/CleanedData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount ('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
from sklearn.utils import resample

# STEP 1: Loading Dataset

# Use the raw URL for the CSV file on GitHub
url = "https://raw.githubusercontent.com/Hushpuppyzac/DLI-Assignment/main/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv"
df = pd.read_csv(url)

# ------------------------------------------------------------------------------

# STEP 2: Data Cleaning

# Strip leading/trailing whitespaces in column names
df.columns = df.columns.str.strip()

# Remove rows with any missing values
df = df.dropna()  # Drop rows with NaN (e.g. 'Flow Bytes/s')

# Drop columns with constant values (no variance)
constant_cols = [col for col in df.columns if df[col].nunique() == 1]
df = df.drop(columns=constant_cols)

# Remove duplicated rows (if any)
df = df.drop_duplicates()

# Encode 'Label' column: 'DDoS' as 1, 'BENIGN' as 0
df['Label'] = df['Label'].apply(lambda x: 1 if x != 'BENIGN' else 0)

# Show cleaned dataset info
print("="*50)
print(" AFTER CLEANING")
print("="*50)
print(f"Total rows after cleaning: {len(df)}")
print("Label distribution after cleaning:")
print(df['Label'].value_counts())
print("="*50)
df.head()

# ------------------------------------------------------------------------------

# STEP 3: Class Balancing with Undersampling


# Separate majority (DDoS) and minority (BENIGN) classes
df_majority = df[df['Label'] == 1]  # DDoS
df_minority = df[df['Label'] == 0]  # BENIGN

print("⚖️ Class Distribution Before Balancing:")
print(df['Label'].value_counts())

# Downsample the majority class (DDoS) to match the number of benign samples
df_majority_downsampled = resample(df_majority,
                                   replace=False,            # sample without replacement
                                   n_samples=len(df_minority),  # match minority count
                                   random_state=42)           # reproducibility

# Combine downsampled majority with minority class
df_balanced = pd.concat([df_majority_downsampled, df_minority])

# Shuffle the combined dataframe
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# 🧾 Show updated class balance
print("\n AFTER UNDERSAMPLING")
print("="*50)
print(f"Total rows after balancing: {len(df_balanced)}")
print("Class balance:")
print(df_balanced['Label'].value_counts())
print("="*50)


# ------------------------------------------------------------------------------

# STEP 3: Display sample data

# Columns to show in sample output
columns_to_show = ['Destination Port', 'Flow Duration', 'Label']

# Count label values
ddos_count = df[df['Label'] == 1].shape[0]
benign_count = df[df['Label'] == 0].shape[0]
total_rows = df.shape[0]

print("\n" + "="*60)
print("SAMPLE OF CLEANED DATAFRAME")
print("="*60)
print(f"Total Rows     : {total_rows}")
print(f"DDoS Attacks   : {ddos_count}")
print(f"Benign Records : {benign_count}")
print("="*60)

# Display sample rows
print("\n Cleaned DataFrame (First 5 Rows):")
display(df[columns_to_show].head(5))

print("\n DDoS Samples (First 5):")
display(df[df['Label'] == 1][columns_to_show].head(5))

print("\n Benign Samples (First 5):")
display(df[df['Label'] == 0][columns_to_show].head(5))

# ------------------------------------------------------------------------------

# STEP 4: Extract Statistical Features from Network Data

# Create a trimmed preview column from flow characteristics (for readability only)
df['Flow Preview'] = df[['Destination Port', 'Flow Duration']].astype(str).agg(' | '.join, axis=1)

# Feature 1: Packet length difference
df['pkt_length_diff'] = df['Max Packet Length'] - df['Min Packet Length']

# Feature 2: Packet length variation (Max / Mean)
df['pkt_length_var_ratio'] = (df['Max Packet Length'] / (df['Packet Length Mean'] + 1e-5)).round(3)

# Feature 3: Byte ratio (Fwd vs Bwd)
df['byte_ratio'] = (df['Total Length of Fwd Packets'] / (df['Total Length of Bwd Packets'] + 1e-5)).round(3)

# Feature 4: Flow duration per packet
df['duration_per_packet'] = (df['Flow Duration'] / (df['Total Fwd Packets'] + df['Total Backward Packets'] + 1e-5)).round(3)

# Feature 5: Average packet size vs Max packet size
df['avg_to_max_ratio'] = (df['Average Packet Size'] / (df['Max Packet Length'] + 1e-5)).round(3)

# Remove duplicated columns (just in case)
df = df.loc[:, ~df.columns.duplicated()]

# Preview extracted features
print("\n Sample Extracted Features (First 5):")
feature_cols = ['Flow Preview', 'pkt_length_diff', 'pkt_length_var_ratio',
                'byte_ratio', 'duration_per_packet', 'avg_to_max_ratio']
display(df[feature_cols].head(5))

# Show final column list
print("\n Final Columns:")
print(df.columns.tolist())


/tmp/ipython-input-2-3211539427.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Label'] = df['Label'].apply(lambda x: 1 if x != 'BENIGN' else 0)


 AFTER CLEANING
Total rows after cleaning: 223108
Label distribution after cleaning:
Label
1    128016
0     95092
Name: count, dtype: int64
⚖️ Class Distribution Before Balancing:
Label
1    128016
0     95092
Name: count, dtype: int64

 AFTER UNDERSAMPLING
Total rows after balancing: 190184
Class balance:
Label
1    95092
0    95092
Name: count, dtype: int64

SAMPLE OF CLEANED DATAFRAME
Total Rows     : 223108
DDoS Attacks   : 128016
Benign Records : 95092

 Cleaned DataFrame (First 5 Rows):


,Destination Port,Flow Duration,Label
0,54865,3,0
1,55054,109,0
2,55055,52,0
3,46236,34,0
4,54863,3,0



 DDoS Samples (First 5):


,Destination Port,Flow Duration,Label
18883,80,1293792,1
18884,80,4421382,1
18885,80,1083538,1
18886,80,80034360,1
18887,80,642654,1



 Benign Samples (First 5):


,Destination Port,Flow Duration,Label
0,54865,3,0
1,55054,109,0
2,55055,52,0
3,46236,34,0
4,54863,3,0



 Sample Extracted Features (First 5):


,Flow Preview,pkt_length_diff,pkt_length_var_ratio,byte_ratio,duration_per_packet,avg_to_max_ratio
0,54865 | 3,0,1.0,1200000.0,1.5,1.5
1,55054 | 109,0,1.0,1.0,54.5,1.5
2,55055 | 52,0,1.0,1.0,26.0,1.5
3,46236 | 34,0,1.0,1.0,17.0,1.5
4,54863 | 3,0,1.0,1200000.0,1.5,1.5



 Final Columns:
['Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'ECE Flag Count', 'Down/Up Ratio', 'Average Packet Size', 'Avg Fwd 